# Accent classification - Russian and English

In [100]:
#Import all the required packages 
# here we will use 3 classifiers (SVC, Naive bayes and random forest) and measure there acccuracy

import os
import pandas as pd
import numpy as np
import librosa
from sklearn.preprocessing import LabelEncoder
from pyAudioAnalysis import audioFeatureExtraction
from sklearn.svm import SVC
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix 
from sklearn.metrics import classification_report 
from sklearn.model_selection import train_test_split
import csv

In [2]:
#Change your directory to project folder 
directory = 'E:/stdies/Data Science/q2/speech processing/project/Accent classification'
 

In [3]:
#creating header for the dataset
header = 'filename '
for i in range(1, 14):
    header += f' mfcc{i}' # creating 13 MFCCs column headers
header += ' label'
header = header.split()

When extracting features from a speech or sound use .WAV format

If you have voices / sounds in .MP3 format you convert that using this code below

from os import path
from pydub import AudioSegment

def convertMP3toWAV():
    print("Start")
    directory = 'Path/to/your/project' #change path to project folder
    pa=directory+'/raw/'
    print(pa)
    for wav_file in os.listdir(pa):
        print(wav_file)
        f=wav_file[:-4]
        dst=f+".wav"
        print(dst)
        sound = AudioSegment.from_mp3(pa+wav_file)
        sound.export(pa+dst, format="wav")
        print("successfully completed")

convertMP3toWAV()

Note: make sure you have all the packages installed for this





In [30]:
# #processing the voice in the directory and extracting the 13th order MFCCs using librosa.
# #Praat and opensmile can also be used
# # file = open('data.csv', 'w', newline='')
# with file:
#     writer = csv.writer(file)
#     writer.writerow(header)
# audio_path = directory+'/wav/' # path to the wav files 
# for wav_file in os.listdir(directory+'/wav/'):
#     print("starting file ",wav_file)
#     s = wav_file
#     s = s[:-4]
#     name_lable = ''.join([i for i in s if not i.isdigit()]) #taking the label for the voice 
#     y, sr = librosa.load(audio_path+wav_file, mono=True, duration=30) # loading the wav files in librosa by taking only 30 seconds
#     librosa.load(audio_path+wav_file, sr=44100) # resampling the frequency of the files to get similar datas
#     mfcc = librosa.feature.mfcc(y=y, sr=sr,n_mfcc=13) # extracting 13 coefficents from MFCCs
#     to_append = f'{wav_file}'    
#     for e in mfcc:
#         to_append += f' {np.mean(e)}' 
#     to_append += f' {name_lable}'
#     file = open('data.csv', 'a', newline='')
#     with file:
#         writer = csv.writer(file)
#         writer.writerow(to_append.split())
#     print("ending file ",wav_file)
# print("Done")

In [43]:
print('Importing CSV....')
data=pd.read_csv(directory + '/data1.csv', low_memory=False)

X = data[['mfcc1', 'mfcc2', 'mfcc3', 'mfcc4', 'mfcc5', 'mfcc6', 'mfcc7', 'mfcc8', 'mfcc9', 'mfcc10', 'mfcc11', 'mfcc12', 'mfcc13']].values
y = data['label'].values

#print(y)

del(data)

print('Done..')

Importing CSV....
Done..


In [95]:
labelencoder_y = LabelEncoder()
y = labelencoder_y.fit_transform(y.ravel())

#print(y) 
#0 is English , 1 is russian

print('Splitting the dataset into the Training set and Test set')

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.32, random_state =1 , shuffle=True)
#print("x train ",X_train)
#print("y train ",y_train)
#print("X test ",X_test)
#print("y test ",y_test)
print('Done..')

Splitting the dataset into the Training set and Test set
Done..


In [96]:
print('Fitting Kernel SVM to the Training set')

classifier = SVC(C=0.01,kernel='linear',gamma=0.3, random_state =1, max_iter=100000)
classifier.fit(X_train, y_train)
#x = np.array(tdata)
#values_new = np.reshape(x, (1, -1))
print("0 - Russian, 1 - English\n")
print("Test labels - ",y_test)
pred = classifier.predict(X_test)
print("Predicted Value - ",pred)
SVCscore = classifier.score(X_test,y_test)

SVCscoretr = classifier.score(X_train,y_train)
print('Score = ',SVCscoretr)

Fitting Kernel SVM to the Training set
0 - Russian, 1 - English

Test labels -  [0 1 1 0 0 1 1 1 1 0 1 1 1 1 1 0 1 1 1 0 0 0 1 1 1 1 0 0 0 0 0 0 1 0 0 1 0
 0 1 0 0 0 0 0 0 1 1 1 0 1 0 1 0 1 0 0 1 0 0 0 1 1 1 0 1 0 0 0 1 1 0 0 0 1
 1 0 1 0 0 0 0 0 1 1 0 0 0 0 1 1 0 1 0 0 0 0 0 0 1 1 0 1 1]
Predicted Value -  [1 1 0 0 0 0 0 1 1 0 1 1 0 1 1 0 0 1 1 1 0 0 0 1 1 0 0 0 0 0 1 0 0 0 1 1 1
 0 0 0 0 0 1 0 0 1 1 1 1 0 0 1 0 0 0 1 0 1 1 1 0 0 1 1 1 0 1 0 0 1 0 0 1 1
 1 0 1 0 0 0 0 1 0 0 0 0 1 0 1 1 1 0 0 1 0 0 0 1 1 0 0 0 1]
Score =  0.7327188940092166


In [97]:
# Random forest 
rfc_clf = RandomForestClassifier(n_estimators=100, bootstrap = True, max_features = 'sqrt')
rfc_clf.fit(X_train, y_train)
rfc_prediction = rfc_clf.predict(X_test)
print("Test labels - ",y_test)
print ("Predicted Value - ",rfc_prediction)
RFscore = rfc_clf.score(X_test,y_test)
#print('Score = ',RFscore)
RFscoretr = rfc_clf.score(X_train,y_train)
print('Score = ',RFscoretr)




Test labels -  [0 1 1 0 0 1 1 1 1 0 1 1 1 1 1 0 1 1 1 0 0 0 1 1 1 1 0 0 0 0 0 0 1 0 0 1 0
 0 1 0 0 0 0 0 0 1 1 1 0 1 0 1 0 1 0 0 1 0 0 0 1 1 1 0 1 0 0 0 1 1 0 0 0 1
 1 0 1 0 0 0 0 0 1 1 0 0 0 0 1 1 0 1 0 0 0 0 0 0 1 1 0 1 1]
Predicted Value -  [0 1 0 0 0 1 0 1 1 0 1 1 1 1 1 0 0 1 1 0 0 0 1 1 0 1 1 0 0 0 0 0 0 0 0 1 0
 0 1 0 0 0 0 0 0 1 1 1 0 0 0 1 0 0 0 0 1 1 0 0 0 1 1 0 1 0 0 0 0 1 0 0 0 0
 1 0 1 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 1 1 0 0 1]
Score =  1.0


In [98]:
# Naive Bayes Classifier 
classifierNB = GaussianNB()
classifierNB.fit(X_train, y_train)
gpred = classifierNB.predict(X_test)
print("Test labels - ",y_test)
print ("Predicted Value - ",gpred)
NBscore = classifierNB.score(X_test,y_test)
#print('Score = ',NBscore)
Bscoretr = classifierNB.score(X_train,y_train)
print('Score = ',Bscoretr)

Test labels -  [0 1 1 0 0 1 1 1 1 0 1 1 1 1 1 0 1 1 1 0 0 0 1 1 1 1 0 0 0 0 0 0 1 0 0 1 0
 0 1 0 0 0 0 0 0 1 1 1 0 1 0 1 0 1 0 0 1 0 0 0 1 1 1 0 1 0 0 0 1 1 0 0 0 1
 1 0 1 0 0 0 0 0 1 1 0 0 0 0 1 1 0 1 0 0 0 0 0 0 1 1 0 1 1]
Predicted Value -  [1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1 1 1 0 1 1 1 0 1 1 1 1 1 1 1 0 1 1 1 0
 0 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0 1 1 0 1 0 0 1 1 0 1 1 1 0
 1 0 1 1 1 1 0 1 0 1 1 1 0 1 1 1 0 1 1 1 1 0 1 0 1 1 1 0 1]
Score =  0.5576036866359447


In [99]:
#finding the best classifier for this problem

classifiers_score = ['SVM', 'Random forest' , 'Naive Bayes']
accuracy_score = np.array([SVCscore, RFscore, NBscore])
print(accuracy_score)
max_acc = np.argmax(accuracy_score)
print(classifiers_score[max_acc] + ' is the best classifier for this problem')


[0.62135922 0.83495146 0.49514563]
Random forest is the best classifier for this problem


In [29]:
# file = open('data.csv', 'w', newline='')
# with file:
#     writer = csv.writer(file)
#     writer.writerow(header)
# audio_path = directory+'/separate_words/' # path to the wav files 
# for wav_file in os.listdir(directory+'/separate_words/'):
#     print("starting file ",wav_file)
#     s = wav_file
#     s = s[:-4]
#     name_lable = ''.join([i for i in s if not i.isdigit()]) #taking the label for the voice 
#     y, sr = librosa.load(audio_path+wav_file, mono=True, duration=30) # loading the wav files in librosa by taking only 30 seconds
#     #librosa.load(audio_path+wav_file, sr=44100) # resampling the frequency of the files to get similar datas
#     mfcc = librosa.feature.mfcc(y=y, sr=sr,n_mfcc=13) # extracting 13 coefficents from MFCCs
#     to_append = f'{wav_file}'    
#     for e in mfcc:
#         to_append += f' {np.mean(e)}' 
#     to_append += f' {name_lable}'
#     file = open('data.csv', 'a', newline='')
#     with file:
#         writer = csv.writer(file)
#         writer.writerow(to_append.split())
#     print("ending file ",wav_file)
# print("Done")

In [104]:
actual = [0 ,1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0,
 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0 ,1 ,1 ,0 ,0, 0, 1,
 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1]

pred = [1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1,
 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1,0, 1, 1, 1, 0, 0, 1, 1, 1, 0 ,1 ,0 ,0, 1, 0, 0, 1, 1,
 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1]
results = confusion_matrix(actual, pred) 
print ('Confusion Matrix :')
print(results) 
print ('Accuracy Score :',accuracy_score(actual, pred))
print ('Accuracy classification report for SVM: ')
print (classification_report(actual, pred))

Confusion Matrix :
[[38 19]
 [20 26]]
Accuracy Score : 0.6213592233009708
Accuracy classification report for SVM: 
              precision    recall  f1-score   support

           0       0.66      0.67      0.66        57
           1       0.58      0.57      0.57        46

   micro avg       0.62      0.62      0.62       103
   macro avg       0.62      0.62      0.62       103
weighted avg       0.62      0.62      0.62       103



In [105]:
actual = [0 ,1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0,
 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0 ,1 ,1 ,0 ,0, 0, 1,
 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1]

pred = [0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0,0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,
 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1]
results = confusion_matrix(actual, pred) 
print ('Confusion Matrix :')
print(results) 
print ('Accuracy Score :',accuracy_score(actual, pred))
print ('Accuracy classification report for Random forest: ')
print (classification_report(actual, pred))

Confusion Matrix :
[[54  3]
 [14 32]]
Accuracy Score : 0.8349514563106796
Accuracy classification report for Random forest: 
              precision    recall  f1-score   support

           0       0.79      0.95      0.86        57
           1       0.91      0.70      0.79        46

   micro avg       0.83      0.83      0.83       103
   macro avg       0.85      0.82      0.83       103
weighted avg       0.85      0.83      0.83       103

